In [1]:
import torch as tt
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook 
from torch.autograd import Variable

In [2]:
import pandas as pd
import numpy as np
import random
import torch.nn.functional as F

In [3]:
import os
import re
from collections import defaultdict, Counter
from string import punctuation
from sklearn.model_selection import train_test_split


punct = punctuation+'«»—…“”*№–'

Основная идея - мы обучаем Tripletloss на текстах газет, тексты, которые принадлежат одной газете должны иметь схожие эмбединги. Далее эти эмбединги в качестве прихнаков мы подаем knn. Даем эмбединги теста и предсказываем результат. Здесь важнешую роль будут иметь эмбединги. Если они плохие - то результат будет соответствующий.

# Data

In [4]:
label = ['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Так как у меня не получалось импортировать из sklearn, я нашла данные в интернете и разделила их на несколько частей. Это словари, где key - индекс газеты из списка выше, а значения - индексы текстов в списке all_texts, где хранятся все-все тексты. 

In [5]:
import pickle

with open('../input/8-hw-s2/all_texts.pickle', 'rb') as f:
    all_texts = pickle.load(f)
    
with open('../input/8-hw-s/x_test.pickle', 'rb') as f:
    x_test = pickle.load(f)
    
with open('../input/8-hw-s/x_train.pickle', 'rb') as f:
    x_train = pickle.load(f)
    
with open('../input/8-hw-s/x_valid.pickle', 'rb') as f:
    x_valid = pickle.load(f)


In [6]:
all_texts[0]

'From: mathew <mathew@mantis.co.uk> Subject: Alt.Atheism FAQ: Atheist Resources Summary: Books, addresses, music -- anything related to atheism Keywords: FAQ, atheism, books, music, fiction, addresses, contacts Expires: Thu, 29 Apr 1993 11:57:19 GMT Distribution: world Organization: Mantis Consultants, Cambridge. UK. Supersedes: <19930301143317@mantis.co.uk> Lines: 290  Archive-name: atheism/resources Alt-atheism-archive-name: resources Last-modified: 11 December 1992 Version: 1.0                                Atheist Resources                        Addresses of Atheist Organizations                                       USA  FREEDOM FROM RELIGION FOUNDATION  Darwin fish bumper stickers and assorted other atheist paraphernalia are available from the Freedom From Religion Foundation in the US.  Write to:  FFRF, P.O. Box 750, Madison, WI 53701. Telephone: (608) 256-8900  EVOLUTION DESIGNS  Evolution Designs sell the "Darwin fish".  It\'s a fish symbol, like the ones Christians stick on

In [7]:
x_train[0][:10]

[23, 252, 447, 56, 476, 376, 227, 474, 292, 157]

In [8]:
x_test[0][:10]

[11314, 11315, 11316, 11317, 11318, 11319, 11320, 11321, 11322, 11323]

Создаем пары pos, neg, anc

In [9]:
def test_set(data):

    x = []
    y = []

    for cls in data:
        for txt in data[cls]:
            x.append(txt)
            y.append(cls)
    
    return x, y


def datasett(data, all_texts):
    
    all_size = [len(data[i]) for i in data]
    #indices_anc = np.random.choice(list(range(len(label))), size= sum(all_size))
    
    indices_anc = np.random.choice(list(range(len(label))), size=sum(all_size))
    
    pair = []
    y = []

    for anc_ind in indices_anc:
    
        neg_ind = np.random.choice([l for l in range(len(label)) if anc_ind != l])
    
        pos, anc = [np.random.choice(data[anc_ind]) for _ in range(2)]
        neg = np.random.choice(data[neg_ind])
    
        pair.append((anc, pos, neg))
        y.append(anc_ind)
    
    return pair, y


train_x, train_y = datasett(x_train, all_texts)
valid_x, valid_y = datasett(x_valid, all_texts)
test_x, test_y = test_set(x_test)

In [10]:
train_x[0], train_y[0]

((3719, 3682, 4166), 6)

Build vocab

In [11]:
def tokenizer(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return words

In [12]:
vocab = Counter()
max_len = 0

for text in all_texts:
    tok = tokenizer(text)
    vocab.update(tok)
    if len(tok) > max_len: max_len = len(tok)


In [13]:
max_len

11821

In [14]:
vocab = [i for i in vocab if vocab[i] > 10]

In [15]:
vocab += ['<pad>', '<eos>']

In [16]:
len(vocab)

22415

In [17]:
vocab.index('from')

0

In [18]:
def tokenizer2(text, vocab):
    
    tensor = tt.zeros(11821).long()
    tensor = tensor.fill_(22413)  # padding
    index = 0

    
    for word in text.lower().split():

        try:
            tensor[index] = vocab.index(word.strip(punct))
            index += 1
        except: continue
       
    tensor[index] = 22414

    return tensor, index

In [19]:
new_text = []
max_len = 0

for text in tqdm_notebook(all_texts):
    tensor, index = tokenizer2(text, vocab)
    new_text.append(tensor) 
    if index > max_len: max_len = index


In [20]:
max_len

11246

In [21]:
new_text = [text[:max_len] for text in new_text]

In [22]:
new_text[0]

tensor([    0,     1,     2,  ..., 22413, 22413, 22413])

In [23]:
del all_texts

# MOdel

In [24]:
def _train_epoch(model, iterator, optimizer, curr_epoch):

    model.train()

    running_loss = 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, batch in enumerate(iterator):
        optimizer.zero_grad()

        loss = model(batch).sum()
        loss.backward()
        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()
        
        loss_smoothing = i / (i+1)
        running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss

        iterator.set_postfix(loss='%.5f' % running_loss)

    return running_loss

def _test_epoch(model, iterator):
    model.eval()
    epoch_loss = 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch in iterator:
            loss = model(batch).sum()
            epoch_loss += loss.data.item()

    return epoch_loss / n_batches


def nn_train(model, train_iterator, valid_iterator, optimizer, n_epochs=100,
          scheduler=None, early_stopping=0):

    prev_loss = 100500
    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss = _train_epoch(model, train_iterator, optimizer, epoch)
        valid_loss = _test_epoch(model, valid_iterator)

        valid_loss = valid_loss
        print('validation loss %.5f' % valid_loss)

        record = {'epoch': epoch, 'train_loss': train_loss, 'valid_loss': valid_loss}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break

            prev_loss = min(prev_loss, valid_loss)

In [25]:
def triplet_loss(anchor_embed, pos_embed, neg_embed):
    return F.cosine_similarity(anchor_embed, neg_embed) - F.cosine_similarity(anchor_embed, pos_embed)


class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size):
        super(MyModel, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_size)
        tt.nn.init.uniform_(self.embedding.weight)
        self.fc = nn.Linear(embed_size, 1)
        
    def branch(self, x):
        x = self.embedding(x)
        x = self.fc(x)
        return x
        
    def forward(self, batch):
        
        anchor = new_text[batch[0]]
        pos = new_text[batch[1]]
        neg =  new_text[batch[2]]
        
        anchor = self.branch(anchor)
        pos = self.branch(pos)
        neg = self.branch(neg)
        
        x = triplet_loss(anchor, pos, neg)
        #x = x.squeeze(1)
        
        return x


Так как мой компьютер не может справиться с Elmo или другими эмбедингами типа 300google, даже если минимально сократить данные, я импользую рандомные веса, что сильно сказывается на качестве. Насколько я понимаю, в данном случае нам бы хотелось, чтобы при обучении loss возрастал, так как это бы означало, что расстояние между anc и neg больше, чем между anc и pos, что нам и нужно. 

In [26]:
batch_size = 32
embed_size = 100

model = MyModel(len(vocab), embed_size)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

nn_train(model, train_x, valid_x, optimizer, n_epochs=3, scheduler=scheduler, early_stopping=2)



validation loss 0.00000



validation loss -1.83827



validation loss 1.48564


# KNN

Собираем данные для кнн.

In [31]:
import annoy


def knn(train_x, ntree, vec_len):

    vector_length = vec_len
    t = annoy.AnnoyIndex(vector_length)

    for i, v in zip(range(len(train_x)), train_x):
        emb = model.branch(new_text[v[0]]).squeeze(1)
        t.add_item(i, emb)
        
    t.build(ntree)
    return t

In [32]:
vec_len = len(model.branch(new_text[0]).squeeze(1))

In [33]:
t = knn(train_x, 10, vec_len)

In [57]:
cos = nn.CosineSimilarity(dim=0)

In [61]:
tt.tensor(t.get_item_vector(11))

tensor([1.3862, 0.4346, 0.8941,  ..., 1.2750, 1.2750, 1.2750])

Так как в формуле loss мы использовали косинусную близость, здесь ее тоже нужно использовать. Насколько я поняла, annoy не дает возможности выбрать коссинусную близость, поэтому мы возьмем какое-то кол-во соседей от annoy и уже посчитаем расстояния до них по нашей метрике.

In [62]:
def predict(test_x, model, n, t):

    pred = []

    for item in tqdm_notebook(test_x):
        
        item = model.branch(new_text[item]).squeeze(1)
    
        res = {}
        for i in t.get_nns_by_vector(item, n):
            res[i] = int(cos(tt.tensor(t.get_item_vector(i)), item))
    
        ind = sorted(res.items(), key=lambda kv: kv[1])
        a = Counter([train_y[i[0]] for i in ind])
        pred.append(max(a, key=a.get))
    
    return pred

In [64]:
pred = predict(test_x, model, 5, t)

In [66]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(test_y, pred)

acc

0.06837493361656931

Результат очень плохой, это равноценно, если бы мы рандомно определяли класс. Это связано с эмбэдингами. Возможно, при использовании Elmo, удастся получить лучший результат, однако у меня такой возможности нет.

То, что мы можем регулировать сейчас - кол-во соседей. Попробуем разные значения.

In [69]:
res = {}

for i in (1, 3, 10, 20):
    pred = predict(test_x, model, i, t)
    acc = accuracy_score(test_y, pred)
    print(i, acc)
    res[i] = acc

1 0.0681093998937865


3 0.07089750398300584


10 0.06983536909187467


KeyboardInterrupt: 

In [70]:
def predict2(test_x, model, n, t, k):

    pred = []

    for item in tqdm_notebook(test_x):
        
        item = model.branch(new_text[item]).squeeze(1)
    
        res = {}
        for i in t.get_nns_by_vector(item, k):
            res[i] = int(cos(tt.tensor(t.get_item_vector(i)), item))
    
        ind = sorted(res.items(), key=lambda kv: kv[1])[:n]
        a = Counter([train_y[i[0]] for i in ind])
        pred.append(max(a, key=a.get))
    
    return pred

In [ ]:
res = {}

for k in (50, 100):
    pred = predict2(test_x, model, k, t, 3)
    acc = accuracy_score(test_y, pred)
    print(k, acc)
    res[k] = acc